# libraries


In [1]:
     
import serial
import time
import cv2
import numpy as np

# modbus to regular


In [2]:
import struct
import time
from pymodbus.client import ModbusSerialClient as ModbusClient

# הגדרות החיבור - ניתן לשינוי
PORT = "COM3"  # שם הפורט - נסה גם COM5 אם לא עובד
BAUDRATE = 115200  # הורדנו מ230400 לערך יותר סטנדרטי
TIMEOUT = 2  # זמן מקסימלי להמתנה לתשובה
SLAVE_ID = 1  # כתובת המכשיר
REGISTER_ADDRESS = 0  # כתובת הרגיסטר
REGISTER_COUNT = 2  # מספר רגיסטרים לקריאה

def create_modbus_client():
    """
    יוצר ומחזיר לקוח Modbus RTU עם הגדרות מותאמות
    """
    client = ModbusClient(
        port=PORT,
        baudrate=BAUDRATE,
        timeout=TIMEOUT,
        parity='N',  # No parity
        stopbits=1,
        bytesize=8,
        strict=False  # פחות מחמיר על שגיאות
    )
    return client

def read_modbus_registers(slave_id=SLAVE_ID, register_address=REGISTER_ADDRESS, count=REGISTER_COUNT):
    """
    קורא רגיסטרים בפרוטוקול MODBUS RTU ומחזיר את הנתונים.
    מטפל בשגיאות ומנסה מספר פעמים במקרה של כישלון.
    """
    client = create_modbus_client()
    
    # נסיון התחברות עם מספר ניסיונות
    max_attempts = 3
    for attempt in range(max_attempts):
        try:
            print(f"🔄 Attempting connection to {PORT} (attempt {attempt + 1}/{max_attempts})...")
            
            if client.connect():
                print(f"✅ Successfully connected to {PORT}")
                break
            else:
                print(f"❌ Failed to connect to {PORT}")
                if attempt < max_attempts - 1:
                    time.sleep(1)  # המתנה קצרה לפני ניסיון נוסף
                continue
                
        except Exception as e:
            print(f"❌ Connection error: {e}")
            if attempt < max_attempts - 1:
                time.sleep(1)
            continue
    else:
        print("❌ Failed to connect after all attempts")
        return None

    try:
        # קריאת רגיסטרים
        print(f"📖 Reading {count} registers from address {register_address}, slave {slave_id}")
        response = client.read_input_registers(
            address=register_address, 
            count=count, 
            slave=slave_id
        )
        
        if response.isError():
            print(f"❌ Modbus Error: {response}")
            return None
            
        print(f"✅ Successfully read {len(response.registers)} registers")
        return response.registers
        
    except Exception as e:
        print(f"❌ Read error: {e}")
        return None
        
    finally:
        try:
            client.close()
            print("🔌 Connection closed")
        except:
            pass

def convert_to_a2d(modbus_values, v_ref=5.0, resolution=16):
    """
    ממיר ערכים שהתקבלו ממודבוס לערכים אנלוגיים (A2D).
    """
    if not modbus_values:
        return []
        
    max_value = 2**resolution - 1  # 65535 עבור 16 ביט
    a2d_values = []
    
    for val in modbus_values:
        # בדיקת תקינות הערך
        if 0 <= val <= max_value:
            voltage = (val / max_value) * v_ref
            a2d_values.append(voltage)
        else:
            print(f"⚠️ Invalid value detected: {val} (expected 0-{max_value})")
            a2d_values.append(0.0)
    
    return a2d_values

def test_modbus_communication():
    """
    פונקציה לבדיקת תקשורת Modbus
    """
    print("=" * 50)
    print("🧪 Testing Modbus Communication")
    print("=" * 50)
    print(f"Port: {PORT}")
    print(f"Baudrate: {BAUDRATE}")
    print(f"Slave ID: {SLAVE_ID}")
    print(f"Register Address: {REGISTER_ADDRESS}")
    print(f"Register Count: {REGISTER_COUNT}")
    print("-" * 50)
    
    # קריאת נתונים
    modbus_data = read_modbus_registers()
    
    if modbus_data:
        print(f"📊 Raw Modbus Data: {modbus_data}")
        
        # המרה לערכי A2D
        a2d_values = convert_to_a2d(modbus_data)
        print(f"🔢 Converted A2D Values (V): {[f'{v:.3f}' for v in a2d_values]}")
        
        # הצגת מידע נוסף
        for i, (raw, voltage) in enumerate(zip(modbus_data, a2d_values)):
            percentage = (voltage / 5.0) * 100
            print(f"   Register {i}: {raw} → {voltage:.3f}V ({percentage:.1f}%)")
            
        return True
    else:
        print("⚠️ Failed to read data. Check:")
        print("   • COM port number and connection")
        print("   • Baud rate settings")
        print("   • Device slave ID")
        print("   • Register address")
        print("   • Physical wiring")
        return False

# הפעלה ידנית - הסר את התגובה כדי להפעיל
# test_modbus_communication()

In [4]:
# Modbus Configuration and Testing

# Try different configurations if the default doesn't work
MODBUS_CONFIGS = [
    {"port": "COM3", "baudrate": 115200, "parity": 'N'},
    {"port": "COM11", "baudrate": 115200, "parity": 'N'},
    {"port": "COM5", "baudrate": 115200, "parity": 'N'},
    {"port": "COM3", "baudrate": 230400, "parity": 'N'},
    {"port": "COM11", "baudrate": 230400, "parity": 'N'},
    {"port": "COM5", "baudrate": 230400, "parity": 'N'},
    {"port": "COM3", "baudrate": 9600, "parity": 'N'},
    {"port": "COM11", "baudrate": 9600, "parity": 'N'},
    {"port": "COM5", "baudrate": 9600, "parity": 'N'},
]

def try_different_configs():
    """
    נסה תצורות שונות של Modbus עד שאחת עובדת
    """
    global PORT, BAUDRATE
    
    for i, config in enumerate(MODBUS_CONFIGS):
        print(f"\n🔧 Trying configuration {i+1}/{len(MODBUS_CONFIGS)}")
        print(f"   Port: {config['port']}, Baudrate: {config['baudrate']}")
        
        # עדכון הגדרות
        PORT = config['port']
        BAUDRATE = config['baudrate']
        
        # ניסיון קריאה
        if test_modbus_communication():
            print(f"✅ Success with configuration: {config}")
            return True
        
        print(f"❌ Configuration {i+1} failed")
    
    print("\n⚠️ All configurations failed. Check hardware connection.")
    return False

def continuous_monitoring(duration_seconds=30):
    """
    ניטור רצוף של נתוני Modbus
    """
    print(f"🔄 Starting continuous monitoring for {duration_seconds} seconds...")
    print("Press Ctrl+C to stop early")
    
    start_time = time.time()
    reading_count = 0
    
    try:
        while time.time() - start_time < duration_seconds:
            modbus_data = read_modbus_registers()
            
            if modbus_data:
                a2d_values = convert_to_a2d(modbus_data)
                reading_count += 1
                
                print(f"Reading {reading_count}: {[f'{v:.3f}V' for v in a2d_values]}")
            else:
                print("❌ Failed to read data")
            
            time.sleep(1)  # קריאה כל שנייה
            
    except KeyboardInterrupt:
        print("\n⏹️ Monitoring stopped by user")
    
    print(f"📊 Completed {reading_count} successful readings")

# 🧪 QUICK TEST - Uncomment ONE of these lines to test:
test_modbus_communication()    # Single test with current settings
# try_different_configs()      # Try multiple configurations automatically
# continuous_monitoring(10)    # Continuous monitoring for 10 seconds

🧪 Testing Modbus Communication
Port: COM3
Baudrate: 115200
Slave ID: 1
Register Address: 0
Register Count: 2
--------------------------------------------------


TypeError: ModbusSerialClient.__init__() got an unexpected keyword argument 'strict'

# main - GAS

In [9]:
#sAGI - WORKING COMMUNICATOR CODE - SIMPLIFIED (Ch1 + Ch3 only)
import datetime
import math

width, height = 500, 500
font = cv2.FONT_HERSHEY_SIMPLEX

def process_channel(channel_bytes):
    swapped_bytes = channel_bytes[2:3] + channel_bytes[1:2] + channel_bytes[0:1]
    value = int.from_bytes(swapped_bytes, byteorder='big')
    return value

def process_packet(packet):
    data_bytes = packet[4:]
    channels = [data_bytes[i:i+3] for i in range(0, 12, 3)]
    return [process_channel(ch) for ch in channels]

def read_serial(port='COM8', baudrate=230400):
    sampels_per_sec = 500
    time_interval = 3  # seconds
    sampels_num = sampels_per_sec * time_interval
 
    c1 = []
    c3 = []
    c1_med = 0
    c3_med = 0
    
    # Calibration values for Ch1 and Ch3
    calibration1 = 1
    calibration3 = 1
    c_mode = 0  # calibration mode flag
    cal1 = 1
    cal3 = 1
    
    # Measurement storage - list of (c1_med, c3_med) tuples
    measurements = []
    measurement_count = 0
    
    no_signal = 0
    
    ser = serial.Serial(port, baudrate, timeout=0.01)
    print(f"Connected to {port} at {baudrate} baud.")
    print("\nInstructions:")
    print("  'c' - Calibrate (at maximum signal position)")
    print("  'a' - Add measurement (stores current Ch1 and Ch3 values)")
    print("  's' - Save all measurements to Excel")
    print("  'q' - Quit")
    
    try:
        while True:
            black_image = np.zeros((height, width, 3), np.uint8)
            
            bytes_available = ser.in_waiting
            if bytes_available >= 16:
                data = ser.read(bytes_available)
                packets = [data[i:i+16] for i in range(len(data)-15) if data[i] == 0xFF]
                
                for packet in packets:
                    channels = process_packet(packet)
                    c1.append(channels[0])  # Channel 1
                    c3.append(channels[2])  # Channel 3
                    
                    if len(c1) > sampels_num:
                        # Convert to numpy arrays for median calculation
                        c1 = np.array(c1)
                        c3 = np.array(c3)
                        
                        # Median of values > 70000 threshold
                        c1_med = np.median(c1[c1 > 70000])
                        c3_med = np.median(c3[c3 > 70000])
                        
                        # Saturation check - zero out if saturated (>8M)
                        if c1_med > 8000000:
                            c1_med = 0
                        if c3_med > 8000000:
                            c3_med = 0
                        
                        # Check for minimum signal (300k threshold)
                        if c1_med < 300000 or c3_med < 300000:
                            no_signal = 1
                        else:
                            no_signal = 0
                        
                        # Normalize channels if calibrated
                        if c_mode == 1:
                            c1R = c1_med / cal1
                            c3R = c3_med / cal3
                            
                            norm13 = max(c1R, c3R)
                            
                            if not np.isnan(norm13) and norm13 != 0:
                                c1_med /= norm13
                                c3_med /= norm13
                                no_signal = 0
                            else:
                                c1_med = c3_med = 0
                                no_signal = 1
                        
                        # Reset buffers for next interval
                        c1 = []
                        c3 = []
                
                # Display update
                black_image = np.zeros((height, width, 3), np.uint8)
                
                # Channel display
                ch1_display = np.round(c1_med / 1e6, 2) if c1_med > 0 else 0
                ch3_display = np.round(c3_med / 1e6, 2) if c3_med > 0 else 0
                
                cv2.putText(black_image, f'Ch1: {ch1_display}M', (50, 100), font, 1.2, (0, 255, 0), 2)
                cv2.putText(black_image, f'Ch3: {ch3_display}M', (50, 150), font, 1.2, (0, 255, 0), 2)
                
                # Status display
                cv2.putText(black_image, f'Measurements: {measurement_count}', (50, 220), font, 0.8, (255, 255, 0), 2)
                
                # Calibration status
                cal_status = "CALIBRATED" if c_mode == 1 else "NOT CALIBRATED"
                cal_color = (0, 255, 0) if c_mode == 1 else (0, 0, 255)
                cv2.putText(black_image, cal_status, (50, 270), font, 0.8, cal_color, 2)
                
                # Signal status
                signal_status = "No Signal" if no_signal == 1 else "Signal OK"
                signal_color = (0, 0, 255) if no_signal == 1 else (0, 255, 0)
                cv2.putText(black_image, signal_status, (50, 320), font, 0.8, signal_color, 2)
                
                # Instructions
                cv2.putText(black_image, "Keys:", (50, 370), font, 0.7, (255, 255, 255), 2)
                cv2.putText(black_image, "'c' - Calibrate", (50, 400), font, 0.6, (255, 255, 255), 1)
                cv2.putText(black_image, "'a' - Add measurement", (50, 425), font, 0.6, (255, 255, 255), 1)
                cv2.putText(black_image, "'s' - Save to Excel", (50, 450), font, 0.6, (255, 255, 255), 1)
                cv2.putText(black_image, "'q' - Quit", (50, 475), font, 0.6, (255, 255, 255), 1)
                
                cv2.imshow('Channel Data (Ch1 + Ch3)', black_image)
                key = cv2.waitKey(1) & 0xFF
                
                if key == ord('q'):
                    break
                    
                elif key == ord('c'):
                    # Calibration - store reference values
                    if c1_med > 100000 and c3_med > 100000:
                        cal1 = c1_med
                        cal3 = c3_med
                        calibration1 = 1
                        calibration3 = 1
                        c_mode = 1
                        print(f"✓ Calibration updated - Ch1: {cal1/1e6:.2f}M, Ch3: {cal3/1e6:.2f}M")
                    else:
                        print("✗ Calibration failed - weak or invalid signal")
                
                elif key == ord('a'):
                    # Add current measurement to list
                    if c1_med > 0 and c3_med > 0:
                        measurements.append((c1_med, c3_med))
                        measurement_count += 1
                        print(f"✓ Measurement #{measurement_count} saved - Ch1: {c1_med/1e6:.2f}M, Ch3: {c3_med/1e6:.2f}M")
                    else:
                        print("✗ Cannot save - no valid signal")
                
                elif key == ord('s'):
                    # Save all measurements to Excel
                    if len(measurements) > 0:
                        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                        save_path = rf"C:\Users\thaim\Downloads\CH1_CH3_measurements_{timestamp}.xlsx"
                        
                        # Create DataFrame with measurements
                        df_data = {
                            'Measurement_Number': list(range(1, len(measurements) + 1)),
                            'Ch1_Raw': [m[0] for m in measurements],
                            'Ch3_Raw': [m[1] for m in measurements],
                            'Ch1_Million': [m[0] / 1e6 for m in measurements],
                            'Ch3_Million': [m[1] / 1e6 for m in measurements]
                        }
                        
                        df = pd.DataFrame(df_data)
                        df.to_excel(save_path, index=False)
                        print(f"✓ Saved {len(measurements)} measurements to: {save_path}")
                        
                        # Clear measurements after saving
                        measurements = []
                        measurement_count = 0
                    else:
                        print("✗ No measurements to save")

    except KeyboardInterrupt:
        print("\nStopped by user.")
        
    finally:
        ser.close()
        cv2.destroyAllWindows()

# Run the simplified sensor reader
read_serial(port='COM8', baudrate=230400)

Connected to COM8 at 230400 baud.

Instructions:
  'c' - Calibrate (at maximum signal position)
  'a' - Add measurement (stores current Ch1 and Ch3 values)
  's' - Save all measurements to Excel
  'q' - Quit
✓ Measurement #1 saved - Ch1: 2.67M, Ch3: 2.14M
✓ Measurement #2 saved - Ch1: 2.69M, Ch3: 2.16M
✓ Measurement #3 saved - Ch1: 2.66M, Ch3: 2.13M
✓ Measurement #4 saved - Ch1: 2.69M, Ch3: 2.16M
✓ Measurement #5 saved - Ch1: 2.65M, Ch3: 2.13M
✓ Measurement #6 saved - Ch1: 2.69M, Ch3: 2.16M
✓ Measurement #7 saved - Ch1: 2.69M, Ch3: 2.15M
✓ Measurement #8 saved - Ch1: 2.67M, Ch3: 2.14M
✓ Measurement #9 saved - Ch1: 2.68M, Ch3: 2.15M
✓ Measurement #10 saved - Ch1: 2.68M, Ch3: 2.16M
✓ Measurement #11 saved - Ch1: 0.72M, Ch3: 0.62M
✓ Measurement #12 saved - Ch1: 0.73M, Ch3: 0.63M
✓ Measurement #13 saved - Ch1: 0.72M, Ch3: 0.62M
✓ Measurement #14 saved - Ch1: 0.73M, Ch3: 0.62M
✓ Measurement #15 saved - Ch1: 0.72M, Ch3: 0.61M
✓ Measurement #16 saved - Ch1: 0.71M, Ch3: 0.61M
✓ Measurement #17

NameError: name 'pd' is not defined